# Python Insights - Analisando Dados com Python

### Case - Cancelamento de Clientes

Você foi contratado por uma empresa com mais de 800 mil clientes para um projeto de Dados. Recentemente a empresa percebeu que da sua base total de clientes, a maioria são clientes inativos, ou seja, que já cancelaram o serviço.

Precisando melhorar seus resultados ela quer conseguir entender os principais motivos desses cancelamentos e quais as ações mais eficientes para reduzir esse número.

Base de dados e arquivos: https://drive.google.com/drive/folders/1uDesZePdkhiraJmiyeZ-w5tfc8XsNYFZ?usp=drive_link

In [7]:
#1: Importar base de dados
import pandas as pd
# Alguns elementos da tabela de dados não são úteis, é necessário descartar os que não interessam para nossa analise (drop)
tabela = pd.read_csv("cancelamentos_sample.csv").drop(columns="CustomerID")

#2: Visualizar base de dados
display(tabela)

idade    sexo  tempo_como_cliente  frequencia_uso  ligacoes_callcenter  \
0       23.0    Male                13.0            22.0                  2.0   
1       49.0    Male                55.0            16.0                  3.0   
2       30.0    Male                 7.0             1.0                  0.0   
3       26.0    Male                40.0             5.0                  3.0   
4       27.0  Female                17.0            30.0                  5.0   
...      ...     ...                 ...             ...                  ...   
49995   62.0  Female                35.0             7.0                  2.0   
49996   36.0    Male                43.0            21.0                  2.0   
49997   55.0    Male                42.0             8.0                  1.0   
49998   40.0  Female                14.0            19.0                  1.0   
49999   64.0    Male                41.0            29.0                  5.0   

       dias_atraso assinatura duracao_contrato  total_gasto  \
0              1.0   Standard           Annual       909.58   
1              6.0    Premium          Monthly       207.00   
2              8.0      Basic           Annual       768.78   
3              8.0    Premium           Annual       398.00   
4              6.0      Basic           Annual       507.00   
...            ...        ...              ...          ...   
49995          8.0      Basic           Annual       232.00   
49996         30.0      Basic        Quarterly       928.00   
49997         12.0      Basic          Monthly       326.00   
49998         17.0    Premium        Quarterly       826.76   
49999          9.0      Basic        Quarterly       700.38   

       meses_ultima_interacao  cancelou  
0                        23.0       0.0  
1                        29.0       1.0  
2                         7.0       0.0  
3                        12.0       1.0  
4                        15.0       1.0  
...                       ...       ...  
49995                    15.0       1.0  
49996                    30.0       1.0  
49997                    27.0       1.0  
49998                    12.0       0.0  
49999                    18.0       1.0  

[50000 rows x 11 columns]

In [8]:
#3: Corrigir problemas na base de dados
# REMOVENDO VALORES VAZIOS
tabela = tabela.dropna()
display(tabela.info())

<class 'pandas.core.frame.DataFrame'>
Index: 49996 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   idade                   49996 non-null  float64
 1   sexo                    49996 non-null  object 
 2   tempo_como_cliente      49996 non-null  float64
 3   frequencia_uso          49996 non-null  float64
 4   ligacoes_callcenter     49996 non-null  float64
 5   dias_atraso             49996 non-null  float64
 6   assinatura              49996 non-null  object 
 7   duracao_contrato        49996 non-null  object 
 8   total_gasto             49996 non-null  float64
 9   meses_ultima_interacao  49996 non-null  float64
 10  cancelou                49996 non-null  float64
dtypes: float64(8), object(3)
memory usage: 4.6+ MB


None

In [10]:

#4: Análise do cancelamento dos clientes (% de clientes que cancelaram)

display(tabela["cancelou"].value_counts()) # Conta quantos cancelaram
display(tabela["cancelou"].value_counts(normalize= True)) # Percentual de cancelamentos


cancelou
1.0    28393
0.0    21603
Name: count, dtype: int64

cancelou
1.0    0.567905
0.0    0.432095
Name: proportion, dtype: float64

In [11]:
#5: Análise da causa do cancelamento: descobrir quais elementos estão influenciando no "cancelou"
#Duracao de contrato:
# display(tabela["duracao_contrato"].value_counts(normalize= True))

# Como descobrir se anual trimestral e mensal influenciam mais? Agrupando a coluna "duracao_contrato" e a "cancelou"

# Calcula media dos valores para saber media de cancelamento do contrato anual, trimestral e mensal:
agrupamento = tabela.groupby("duracao_contrato").mean(numeric_only = True) # Calculando a média apenas dos valores numéricos
display(agrupamento)

# Conclusões: 
# Anual : 46% // Mensal: 100% // Trimestral: 46%
# Sugestão: Oferecer desconto nos contratos anuais e trimestrais
# Já que sabemos que mensal é 100% cancelado, podemos eliminá-lo da análise 
tabela = tabela[tabela["duracao_contrato"] != "Monthly"] # Filtro
display(tabela["cancelou"].value_counts(normalize= True))


idade  tempo_como_cliente  frequencia_uso  \
duracao_contrato                                                  
Annual            38.783985           31.416452       15.910449   
Monthly           41.428470           30.677964       15.577600   
Quarterly         38.833935           31.522099       15.842504   

                  ligacoes_callcenter  dias_atraso  total_gasto  \
duracao_contrato                                                  
Annual                       3.277585    12.533985   650.925840   
Monthly                      4.923917    15.078814   547.508921   
Quarterly                    3.256113    12.480006   654.102443   

                  meses_ultima_interacao  cancelou  
duracao_contrato                                    
Annual                         14.231544  0.464080  
Monthly                        15.392655  1.000000  
Quarterly                      14.364602  0.458759

cancelou
0.0    0.538567
1.0    0.461433
Name: proportion, dtype: float64

In [22]:
import plotly.express as px
import pandas as pd

tabela = pd.read_csv("cancelamentos_sample.csv").drop(columns="CustomerID")

for coluna in tabela.columns:
    grafico = px.histogram(tabela, x = coluna, color = "cancelou")
   
# A PARTIR DOS 50 ANOS 100% CANCELA

In [14]:
# A partir de 21 dias de atraso cancelam
# A partir de 4 ligacoes de callcenter, cancelam
# A partir dos 51 anos, todos cancelam

# Remover da analise os dias de atraso, clientes partir de 4 ligacoes do callcenter e acima dos 50
tabela = tabela[tabela["ligacoes_callcenter"] <= 4]
tabela = tabela[tabela["dias_atraso"] <= 20]
tabela = tabela[tabela["idade"] <= 50]

display(tabela["cancelou"].value_counts(normalize = True))
# Ou seja: essa seria o percentual de cancelamentos caso os problemas apontados acima sejam resolvidos

cancelou
0.0    0.807598
1.0    0.192402
Name: proportion, dtype: float64

# Conclusão:
## Os maiores responsáveis pelas altas taxas de cancelamento são:
- Assinantes mensais
- Usuários que receberam a 4 ligacoes do call center ou mais
- Assinantes a partir dos 51 anos
- Usuários que demoram a partir de 21 dias para resolver o atraso no pagamento

## Ao resolver tais problemas, a taxa de cancelamento iria de 56% para 19%